In [90]:
import json
import pandas as pd
import h3

df = pd.read_csv('macrozones.csv', dtype=str)
df = df.reset_index()

In [91]:
zones = {
    "COLOMBIA": {},
    "MEXICO": {},
    "BRASIL": {},
    "ECUADOR": {}
}

distinct_pc = set()

for i in zones:
    sub_df = df[df['source_country'] == i]
    macrozones = sub_df['macrozone'].unique()
    zones[i] = { macro: {} for macro in macrozones }

    for j in macrozones:
        sub_df_2 = sub_df[sub_df['macrozone'] == j]
        microzones = sub_df_2['microzone'].unique()
        zones[i][j] = { micro: {} for micro in microzones}

        for k in microzones:
            sub_df_3 = sub_df_2[sub_df_2['microzone'] == k]
            postal_codes = sub_df_3['postal_code'].unique()
            zones[i][j][k] = list(postal_codes)
            distinct_pc.update(list(postal_codes))

In [92]:
postal_codes_h3 = {}

mex_files = ['CDMX.geojson', 'JALISCO.geojson', 'MEXICO.geojson', 'COLOMBIA.geojson']


for file in mex_files:
    with open(file, 'r') as fp:
        geojson_file = json.load(fp)

    for feature in geojson_file['features']:
        try:
            pc = feature['properties']['CP']
        except:
            pc = feature['properties']['Codigo_Pos']
        if pc in distinct_pc:
            postal_codes_h3[pc] = []
            if feature['geometry']['type'] == 'GeometryCollection':
                for geo in feature['geometry']['geometries']:
                    postal_codes_h3[pc].extend(list(h3.polyfill(geo, 8, True)))
                postal_codes_h3[pc] = list(set(postal_codes_h3[pc]))

            elif feature['geometry']['type'] == 'MultiPolygon':
                for geo in feature['geometry']['coordinates']:
                    geo_type = {
                        "type": 'Polygon',
                        "coordinates": geo
                    }
                    postal_codes_h3[pc].extend(list(h3.polyfill(geo_type, 8, True)))
                postal_codes_h3[pc] = list(set(postal_codes_h3[pc]))

            else:
                geo = feature['geometry']
                postal_codes_h3[pc].extend(list(h3.polyfill(geo, 8, True)))
                postal_codes_h3[pc] = list(set(postal_codes_h3[pc]))

In [93]:
final_csv = []
cant_find = 0
yes_find = 0
for country in zones:
    for macrozone in zones[country]:
        for microzone in zones[country][macrozone]:
            for postal_code in zones[country][macrozone][microzone]:
                try:
                    yes_find += 1
                    for h3_cell in postal_codes_h3[postal_code]:
                        final_csv.append({
                            "source_country":country,
                            "macrozone": macrozone,
                            "microzone": microzone,
                            "r8_h3": h3_cell
                        })
                except:
                    cant_find += 1

df = pd.DataFrame.from_dict(final_csv)
df.to_csv('result_3.csv')

In [8]:
# Migration for Sao Paulo
import h3
import json
import pandas as pd

with open('SaoPaulo.geojson', 'r') as fp:
    geojson_file = json.load(fp)

final_csv = []

for feature in geojson_file['features']:
    geo = feature['geometry']
    macrozone = feature['properties']['name']
    microzone = feature['properties']['name']

    for idx, point in enumerate(geo['coordinates'][0]):
        point = point[:-1]
        geo['coordinates'][0][idx] = point

    cells = list(h3.polyfill(geo, 8, True))
    for h3_cell in cells:
        final_csv.append({
            "source_country":'BRASIL',
            "macrozone": macrozone,
            "microzone": microzone,
            "r8_h3": h3_cell
        })

df = pd.DataFrame.from_dict(final_csv)
df.to_csv('result_4_r8_SaoPaulo.csv')


In [10]:
import h3
import json
import pandas as pd

final_csv = []
with open('colombia_dep_mun.geojson', 'r') as fp:
    data = json.load(fp)

for feature in data['features']:
    department_code = feature['properties']['DPTO_CCDGO']
    city_code = feature['properties']['MPIO_CCDGO']
    department_name = feature['properties']['DPTO_CNMBR']
    city_name = feature['properties']['MPIO_CNMBR']
    
    if feature['geometry']['type'] == 'Polygon':
        geo = feature['geometry']
        h3_cells = list(h3.polyfill(geo, 9, True))
        for cell in h3_cells:
            final_csv.append({
                "department_code": department_code,
                "city_code": city_code,
                "department_name": department_name,
                "city_name": city_name,
                "r9_h3": cell
            })
    elif feature['geometry']['type'] == 'MultiPolygon':
        for geo_coors in feature['geometry']['coordinates']:    
            geo = {
                "type": 'Polygon',
                'coordinates': geo_coors
            }
            h3_cells = list(h3.polyfill(geo, 9, True))
            for cell in h3_cells:
                final_csv.append({
                    "department_code": department_code,
                    "city_code": city_code,
                    "department_name": department_name,
                    "city_name": city_name,
                    "r9_h3": cell
                })

df = pd.DataFrame.from_dict(final_csv)
df.set_index('r9_h3')
df.to_csv('r9h3_department_city.csv')

In [9]:
df = pd.DataFrame.from_dict(final_csv)
df = df.set_index('r9_h3')
df.to_csv('r9h3_department_city.csv')

,department_code,city_code,department_name,city_name
r9_h3,,,,
896618ad903ffff,05,001,MEDELLÍN,MEDELLÍN
89661837423ffff,05,001,MEDELLÍN,MEDELLÍN
896619d9d3bffff,05,001,MEDELLÍN,MEDELLÍN
896619da27bffff,05,001,MEDELLÍN,MEDELLÍN
89661836c83ffff,05,001,MEDELLÍN,MEDELLÍN
...,...,...,...,...
896644393bbffff,99,773,CUMARIBO,CUMARIBO
896646b3147ffff,99,773,CUMARIBO,CUMARIBO
89664246c03ffff,99,773,CUMARIBO,CUMARIBO
